In [1]:
import numpy
print("numpy:   " + numpy.version.version)

import sklearn
import sklearn.metrics
print("sklearn: " + sklearn.__version__)

numpy:   1.19.2
sklearn: 0.23.2


In [2]:
embeddingsDir = "/home/eml4u/EML4U/data/interim-results/"

idAmericanA = "20100408-american-films"
idAmericanB = "20201101-american-films"
idBritishA = "20100408-british-films"
idBritishB = "20201101-british-films"
idIndianA = "20100408-indian-films"
idIndianB = "20201101-indian-films"

In [3]:
# Load text files
def getTextfile(dir, id, printfile = True):
    file = dir + id + ".txt"
    if printfile:
        print(file)
    return file

fileEmbeddingsAmericanA = getTextfile(embeddingsDir, idAmericanA)
fileEmbeddingsAmericanB = getTextfile(embeddingsDir, idAmericanB)
fileEmbeddingsBritishA  = getTextfile(embeddingsDir, idBritishA)
fileEmbeddingsBritishB  = getTextfile(embeddingsDir, idBritishB)
fileEmbeddingsIndianA  = getTextfile(embeddingsDir, idIndianA)
fileEmbeddingsIndianB  = getTextfile(embeddingsDir, idIndianB)

/home/eml4u/EML4U/data/interim-results/20100408-american-films.txt
/home/eml4u/EML4U/data/interim-results/20201101-american-films.txt
/home/eml4u/EML4U/data/interim-results/20100408-british-films.txt
/home/eml4u/EML4U/data/interim-results/20201101-british-films.txt
/home/eml4u/EML4U/data/interim-results/20100408-indian-films.txt
/home/eml4u/EML4U/data/interim-results/20201101-indian-films.txt


In [4]:
# Load embeddings
def loadEmbeddings(file, note = "", printinfo = True):
    embeddings = numpy.loadtxt(file)
    if printinfo:
        print(str(embeddings.shape) + " " + note)
    return embeddings

embeddingsAmericanA = loadEmbeddings(fileEmbeddingsAmericanA, "AmericanA")
embeddingsAmericanB = loadEmbeddings(fileEmbeddingsAmericanB, "AmericanB")
embeddingsBritishA  = loadEmbeddings(fileEmbeddingsBritishA, "BritishA")
embeddingsBritishB  = loadEmbeddings(fileEmbeddingsBritishB, "BritishB")
embeddingsIndianA  = loadEmbeddings(fileEmbeddingsIndianA, "IndianA")
embeddingsIndianB  = loadEmbeddings(fileEmbeddingsIndianB, "IndianB")

(1000, 768) AmericanA
(1000, 768) AmericanB
(2147, 768) BritishA
(2147, 768) BritishB
(3596, 768) IndianA
(3596, 768) IndianB


In [5]:
# Compute mean
# https://numpy.org/doc/1.19/reference/generated/numpy.mean.html?highlight=mean#numpy.mean
# https://numpy.org/doc/1.19/reference/generated/numpy.reshape.html?highlight=reshape#numpy.reshape
def getMean(embeddings, dimension, note = "", printinfo = True):
    mean = numpy.mean(embeddings, axis=0)
    reshaped = mean.reshape(1, dimension)
    if printinfo:
        #print(str(mean.shape) + " " + note)
        print(str(reshaped.shape) + " " + note)
    return embeddings

meanAmericanA = getMean(embeddingsAmericanA, 768, "AmericanA")
meanAmericanB = getMean(embeddingsAmericanB, 768, "AmericanB")
meanBritishA  = getMean(embeddingsBritishA, 768, "BritishA")
meanBritishB  = getMean(embeddingsBritishB, 768, "BritishB")
meanIndianA  = getMean(embeddingsIndianA, 768, "IndianA")
meanIndianB  = getMean(embeddingsIndianB, 768, "IndianB")

(1, 768) AmericanA
(1, 768) AmericanB
(1, 768) BritishA
(1, 768) BritishB
(1, 768) IndianA
(1, 768) IndianB


In [6]:
# Cosine similarity
# https://scikit-learn.org/0.23/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity
def getPairwiseCosineSimilarity(a, b, note = "", printinfo = True):
    cosSim = sklearn.metrics.pairwise.cosine_similarity(a.reshape(1, -1), b.reshape(1, -1), dense_output=True)[0][0]
    if printinfo:
        print(str(cosSim) + " " + note)
    return cosSim

getPairwiseCosineSimilarity(meanAmericanA, meanAmericanB, "American")
getPairwiseCosineSimilarity(meanBritishA, meanBritishB, "British")
getPairwiseCosineSimilarity(meanIndianA, meanIndianB, "Indian")
print()

0.9470747593129083 American
0.9422465618828457 British
0.931203987093214 Indian



In [7]:
# Cosine similarity single
def getCosineSimilarity(a, b, indexA, indexB, note = "", printinfo = True):
    cosSim = sklearn.metrics.pairwise.cosine_similarity(a[indexA].reshape(1, -1), b[indexB].reshape(1, -1), dense_output=True)[0][0]
    if printinfo:
        print(str(cosSim) + " " + note)
    return cosSim

getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 0, 0, "American 0")
getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 1, 1, "American 1")
getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 10, 10, "American 10")
getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 100, 100, "American 100")
getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 500, 500, "American 500")
getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 600, 600, "American 600")
print()
getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 0, 0, "British 0")
getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 1, 1, "British 1")
getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 10, 10, "British 10")
getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 100, 100, "British 100")
getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 500, 500, "British 500")
getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 600, 600, "British 600")
print()
getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 0, 0, "Indian 0")
getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 1, 1, "Indian 1")
getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 10, 10, "Indian 10")
getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 100, 100, "Indian 100")
getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 500, 500, "Indian 500")
getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 600, 600, "Indian 600")
print()

0.9525117010449139 American 0
0.9715444013842313 American 1
0.9724710753091028 American 10
0.9772824870224932 American 100
0.9393622073417478 American 500
0.9867714675621426 American 600

0.9618288510169046 British 0
0.9325135174538641 British 1
0.9473581782691437 British 10
0.9680476495788866 British 100
0.9310833202893275 British 500
0.8651356315148098 British 600

0.9760882993008546 Indian 0
0.9805102425307188 Indian 1
0.7447943708902642 Indian 10
0.6380358748619926 Indian 100
0.8617119348355946 Indian 500
0.8981347803389674 Indian 600



In [8]:
print(type(embeddingsAmericanA))
print(embeddingsAmericanA.shape)

testEmbeddingsA = embeddingsAmericanA[:3]
#print(type(testEmbeddingsA))
print(testEmbeddingsA.shape)

testEmbeddingsB = embeddingsAmericanB[:3]
#print(type(testEmbeddingsB))
print(testEmbeddingsB.shape)
print()

x1 = getCosineSimilarity(testEmbeddingsA, testEmbeddingsB, 0, 0, "test 0")
x2 = getCosineSimilarity(testEmbeddingsA, testEmbeddingsB, 1, 1, "test 1")
x3 = getCosineSimilarity(testEmbeddingsA, testEmbeddingsB, 2, 2, "test 2")
print()

meanTestA = getMean(testEmbeddingsA, 768, "testA")
meanTestB = getMean(testEmbeddingsB, 768, "testB")
result = getPairwiseCosineSimilarity(meanTestA, meanTestB, "test")
expected = (x1 + x2 + x3) / 3
print(expected)

if expected != result:
    print("TODO")

<class 'numpy.ndarray'>
(1000, 768)
(3, 768)
(3, 768)

0.9525117010449139 test 0
0.9715444013842313 test 1
0.9286622417569126 test 2

(1, 768) testA
(1, 768) testB
0.9513856700392087 test
0.9509061147286859
TODO
